# 모델 저장과 복원
---
- 방법1) 모델 전체 저장 (모델 구조 + 가중치 + 변수)
    * save_model()
    * 복원시 추가 작업 필요 없이 모델 파일 로딩해서 사용 가능
- 방법2) 가중치만 저장
    * save_weight()
    * 복원시 모델 구조 생성 후 가중치 적용
- 방법3) 모델 정체 또는 가중치 자동 저장
    * fit()에서 ModelCheckPoint Event에 대한 callback 등록
    * save_best_only, save_weight_only
- 파일 또는 폴더
    * 파일 확장자가 없으면 폴더로 저장
    * 파일 확장자
        - h5, hdf5 : HDF5 포맷으로 저장
        - ckpt : 체크포인트 저장

In [110]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Dropout

## [2] 데이터 준비 및 로딩

In [111]:
iris = load_iris()

iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [112]:
X = iris['data']
y = iris['target']

In [113]:
print(X.shape, y.shape)

(150, 4) (150,)


In [114]:
print(iris["feature_names"])
print(iris["target_names"])

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


## [3] 데이터 전처리

### [3-1] 스케일링

In [115]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

### [3-2] 데이터 나누기

In [116]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, stratify=y, random_state=42
)

## [4] 모델 구성

In [117]:
model = Sequential()

model.add(Dense(10, activation="relu", input_shape=(4,)))
model.add(Dropout(0.2))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(30, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(3, activation="softmax"))

In [118]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 10)                50        
                                                                 
 dropout_16 (Dropout)        (None, 10)                0         
                                                                 
 dense_30 (Dense)            (None, 20)                220       
                                                                 
 dropout_17 (Dropout)        (None, 20)                0         
                                                                 
 dense_31 (Dense)            (None, 30)                630       
                                                                 
 dropout_18 (Dropout)        (None, 30)                0         
                                                                 
 dense_32 (Dense)            (None, 3)                

## [5] 모델 생성

In [119]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

In [120]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

stop_cb = EarlyStopping(
    patience=10
)

## [6] 모델 학습

In [121]:
model_his = model.fit(
    X_train,
    y_train,
    epochs=500,
    callbacks=[stop_cb],
    validation_split=0.2
)

Epoch 1/500
3/3 [==============================] - 1s 82ms/step - loss: 1.1127 - accuracy: 0.3820 - val_loss: 1.1046 - val_accuracy: 0.2609
Epoch 2/500
3/3 [==============================] - 0s 14ms/step - loss: 1.1035 - accuracy: 0.2809 - val_loss: 1.0908 - val_accuracy: 0.5652
Epoch 3/500
3/3 [==============================] - 0s 13ms/step - loss: 1.0729 - accuracy: 0.4494 - val_loss: 1.0772 - val_accuracy: 0.5652
Epoch 4/500
3/3 [==============================] - 0s 13ms/step - loss: 1.0455 - accuracy: 0.5169 - val_loss: 1.0632 - val_accuracy: 0.6087
Epoch 5/500
3/3 [==============================] - 0s 15ms/step - loss: 1.0565 - accuracy: 0.4494 - val_loss: 1.0485 - val_accuracy: 0.6087
Epoch 6/500
3/3 [==============================] - 0s 16ms/step - loss: 1.0368 - accuracy: 0.5281 - val_loss: 1.0334 - val_accuracy: 0.6522
Epoch 7/500
3/3 [==============================] - 0s 16ms/step - loss: 1.0076 - accuracy: 0.6517 - val_loss: 1.0182 - val_accuracy: 0.6087
Epoch 8/500
3/3 [===

## [7] 모델 평가

In [126]:
loss, accv = model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 0.3097 - accuracy: 0.8684


In [138]:
model_path = "/content/drive/MyDrive/Colab Notebooks/Models/ex-iris"
full_path = model_path + "/ex-iris.h5"
weights_path = model_path + "/weight.h5"

In [136]:
from keras.models import save_model

save_model(
    model,
    filepath=full_path,
)

In [141]:
model.save_weights(weights_path)